In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
print('Dependencies Imported')

Dependencies Imported


In [2]:
!git clone https://github.com/matzim95/ML-datasets

Cloning into 'ML-datasets'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 194 (delta 33), reused 124 (delta 24), pack-reused 60
Receiving objects: 100% (194/194), 35.45 MiB | 11.30 MiB/s, done.
Resolving deltas: 100% (45/45), done.
Checking out files: 100% (127/127), done.


In [3]:
import pandas as pd

def load_dataset(filename, class_column, index_col=None):
    dataset = pd.read_csv(f'ML-datasets/{filename}.csv', index_col=index_col)
    dataset['class'] = dataset[class_column].astype('category').cat.codes
    classes = dataset.pop(class_column).unique()
    return dataset, classes

In [4]:
#Ładowanie danych
titanic, titanic_classes = load_dataset('titanic', 'Survived')
print(titanic_classes)
#Podział na dane treningowe i etykiety
y = titanic.pop('class')
X = titanic

[0 1]


In [5]:
#Przetwarzanie danych
#Sprawdzenie ile mamy danych pusty

titanic.isnull().sum()


PassengerId      0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
#Zamiana wartości pustych 
titanic['Age'].fillna(titanic['Age'].mean(),inplace=True)
#Zamiana zmiennej ciągłej na zmienną przedziałową
titanic['age_bins'] = pd.cut(x=titanic['Age'], bins=[5, 15, 20, 29, 39, 49, 59, 85])

In [7]:
#Wybór zmiennych do algorytmu
titanic = titanic.iloc[:, [1,3,5,6,8,10,11]]

In [8]:
titanic['age_bins']

0      (20, 29]
1      (29, 39]
2      (20, 29]
3      (29, 39]
4      (29, 39]
         ...   
886    (20, 29]
887    (15, 20]
888    (29, 39]
889    (20, 29]
890    (29, 39]
Name: age_bins, Length: 891, dtype: category
Categories (7, interval[int64, right]): [(5, 15] < (15, 20] < (20, 29] < (29, 39] < (39, 49] <
                                         (49, 59] < (59, 85]]

In [9]:
#Zamiana wartości pustych
titanic['age_bins']= titanic['age_bins'].astype('string')
titanic['age_bins'].fillna('9', inplace=True)
titanic['age_bins'] =titanic['age_bins'].astype('category')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [15]:
titanic.describe()

,Pclass,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.523008,0.381594,32.204208
std,0.836071,1.102743,0.806057,49.693429
min,1.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,7.910400
50%,3.000000,0.000000,0.000000,14.454200
75%,3.000000,1.000000,0.000000,31.000000
max,3.000000,8.000000,6.000000,512.329200


In [10]:
#Zamiana danych głównie tekstowych na wartości
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # zamienia tekst na liczby
t3 = titanic.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')
X_d = t3
X_d

,Pclass,Sex,SibSp,Parch,Fare,Embarked,age_bins
0,2,1,1,0,174,2,1
1,0,0,1,0,199,0,2
2,2,0,0,0,197,2,1
3,0,0,1,0,142,2,2
4,2,1,0,0,211,2,2
...,...,...,...,...,...,...,...
886,1,1,0,0,21,2,1
887,0,0,0,0,103,2,0
888,2,0,1,2,78,2,2
889,0,1,0,0,103,0,1


In [31]:
X_d['age_bins'].tail()

886    1
887    0
888    2
889    1
890    2
Name: age_bins, dtype: int64

In [20]:
titanic['Sex']

0        male
1      female
2      female
3      female
4        male
        ...  
886      male
887    female
888    female
889      male
890      male
Name: Sex, Length: 891, dtype: object

In [12]:
y.value_counts()

0    549
1    342
Name: class, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
 X_d, y, test_size=0.33, random_state=42, stratify=y)

In [16]:
#model selection
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
#regressor = AdaBoostClassifier(n_estimators=200)
#regressor.fit(X_train,y_train)
pipe = Pipeline([
    ('scale',StandardScaler()),
    ('gb',GradientBoostingClassifier())  
])
# Parameters of pipelines can be set using '__' separated parameter names:
param_grid = {

    "gb__n_estimators": np.arange(1,50),
}
search = GridSearchCV(pipe, param_grid, scoring = 'f1', n_jobs=-1)
search.fit(X_train, y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.767):
{'gb__n_estimators': 29}


zad.1.
a. Dokonaj procesu dokładnej analizy danych (statystyki, wizualizacje i na jej podstawie wybierz te cechy które rzeczywiście mają znaczenie dla algorytmu - kolumny nie wnoszące dużo informacji możesz pominąć (jezeli nie masz pomysłu użyj skryptu do selekcji cech ze zjazdu zaawansowane problemy klasyfikacji).
b. Wykonaj 2 metody uczenia maszynowego  wybieracjac hiperparametry z Grid Search CV skalujacac rowniez w pipeline, dla lepszej meotdy dokonaj cross walidacji. Zapisz wynik jakie widzisz problemy?